In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install keras


In [ ]:
pip install --upgrade keras


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization, Activation ,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop

In [ ]:
from tensorflow.keras.models import Sequential


In [ ]:
picture_size=48
folder_path="/kaggle/input/face-expression-recognition-dataset/images"

In [ ]:
picture_size=48
folder_path="/kaggle/input/face-expression-recognition-dataset/images"
expression='disgust'
plt.figure(figsize=(12,12))
for i in range(1,10,1):
    plt.subplot(3,3,i)
    img=load_img(folder_path+"/train/"+expression+"/"+
                os.listdir(folder_path+"/train/"+expression)[i],target_size=(picture_size,picture_size))
    plt.imshow(img)
plt.show()


In [ ]:
batch_size=125
datagen_train=ImageDataGenerator()
datagen_val=ImageDataGenerator()
train_set=datagen_train.flow_from_directory(folder_path+"/train",
                                      target_size=(picture_size,picture_size),
                                      color_mode="grayscale",
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                      shuffle=True)
test_set=datagen_val.flow_from_directory(folder_path+"/validation",
                                      target_size=(picture_size,picture_size),
                                      color_mode="grayscale",
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                      shuffle=False)


In [ ]:
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization, Activation ,MaxPooling2D
from keras.optimizers import Adam,SGD,RMSprop
no_of_classes=7
model=Sequential()
# 1st CNN layer
model.add(Conv2D(64,(3,3),padding='same',input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# 2nd CNN layer
model.add(Conv2D(128,(5,5),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# 3rd CNN layer
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
# 4th CNN layer
model.add(Conv2D(512,(3,3),padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())

# Fully Connected 1st Layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
# Fully Connected 2nd Layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes,activation='softmax'))

opt=Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()





In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
checkpoint=ModelCheckpoint("./model.h5",monitor='vcal_acc',verbose=1,save_best_only=True,mode='max')
early_stopping=EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=3,
                            verbose=1,
                            restore_best_weights=True)
reduce_learningrate=ReduceLROnPlateau(monitor='val_loss',
                                      min_delta=0.0001,
                                      factor=0.2,
                                      verbose=1,
                                      patience=3)
                                      
callbacks_list=[early_stopping,checkpoint,reduce_learningrate]
epochs=48

In [ ]:
batch_size=125
datagen_train=ImageDataGenerator()
datagen_val=ImageDataGenerator()
train_set=datagen_train.flow_from_directory(folder_path+"/train",
                                      target_size=(picture_size,picture_size),
                                      color_mode="grayscale",
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                      shuffle=True)
test_set=datagen_val.flow_from_directory(folder_path+"/validation",
                                      target_size=(picture_size,picture_size),
                                      color_mode="grayscale",
                                      batch_size=batch_size,
                                      class_mode='categorical',
                                      shuffle=False)

In [ ]:
history=model.fit_generator(generator=train_set,
                          steps_per_epoch=train_set.n//train_set.batch_size,
                          epochs=epochs,
                          validation_data=test_set,
                          validation_steps=test_set.n//test_set.batch_size,
                          callbacks=callbacks_list)



In [ ]:
model.save('model.h5')

In [ ]:
import shutil
shutil.move('model.h5','/kaggle/working/model.h5')